In [ ]:
# %pip install tfx==1.4.0

In [4]:
import os

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [5]:
context = InteractiveContext(pipeline_root='../tfx')

In [6]:
data_dir = "../data"
example_gen = tfx.components.CsvExampleGen(input_base=os.path.join(os.getcwd(), data_dir))

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "../tfx/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:78956236,xor_checksum:1637688630,sum_checksum:1637688630"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [8]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "../tfx/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [9]:
context.show(statistics_gen.outputs['statistics'])

In [10]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

I1123 09:44:23.864810 506195456 rdbms_metadata_access_object.cc:686] No property is defined for the Type


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "../tfx/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'consumer_complaint_narrative',BYTES,required,,-
'consumer_disputed',INT,required,,-
'issue',STRING,required,,'issue'
'product',STRING,required,,'product'
'state',STRING,required,,'state'
'sub_issue',STRING,required,,'sub_issue'
'sub_product',STRING,required,,'sub_product'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'Untimely response'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender damaged or destroyed vehicle', 'Lender repossessed or sold the vehicle', 'Lender sold the property', 'Loan modification,collection,foreclosure', 'Loan servicing, payments, escrow account', 'Lost or stolen check', 'Lost or stolen money order', 'Making/receiving payments, sending money', 'Managing the line of credit', 'Managing the loan or lease', 'Managing, opening, or closing account', 'Money was not available when promised', 'Other', 'Other fee', 'Other service issues', 'Other transaction issues', 'Overdraft, savings or rewards features', 'Overlimit fee', 'Payment to acct not credited', 'Payoff process', 'Privacy', 'Problems caused by my funds being low', 'Problems when you are unable to pay', 'Received a loan I didn\'t apply for', 'Rewards', 'Sale of account', 'Settlement process and costs', 'Shopping for a line of credit', 'Shopping for a loan or lease', 'Taking out the loan or lease', 'Taking/threatening an illegal action', 'Transaction issue', 'Unable to get credit report/credit score', 'Unauthorized transactions/trans. issues', 'Unexpected/Other fees', 'Unsolicited issuance of credit card', 'Using a debit or ATM card', 'Wrong amount charged or received', 'Incorrect exchange rate'"
'product',"'Bank account or service', 'Consumer Loan', 'Credit card', 'Credit reporting', 'Debt collection', 'Money transfers', 'Mortgage', 'Other financial service', 'Payday loan', 'Prepaid card', 'Student loan'"
'state',"'AA', 'AE', 'AK', 'AL', 'AP', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'FM', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'"
'sub_issue',"'Account status', 'Account terms', 'Account terms and changes', 'Applied for loan/did not receive money', 'Attempted to collect wrong amount', 'Attempted to/Collected exempt funds', 'Billing dispute', 'Called after sent written cease of comm', 'Called outside of 8am-9pm', 'Can\'t contact lender', 'Can\'t decrease my monthly payments', 'Can\'t get flexible payment options', 'Can\'t qualify for a loan', 'Can\'t stop charges to bank account', 'Can\'t temporarily postpone payments', 'Charged bank 

In [12]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

In [13]:
transform_file = os.path.join(os.getcwd(), '../components/module.py')

In [23]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=transform_file)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying keras_trainer.py -> build/lib
copying module_test.py -> build/lib
copying transform.py -> build/lib
copying module.py -> build/lib
installing to /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmpjggvlesc
running install
running install_lib
copying build/lib/keras_trainer.py -> /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmpjggvlesc
copying build/lib/module_test.py -> /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmpjggvlesc
copying build/lib/transform.py -> /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmpjggvlesc
copying build/lib/module.py -> /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmpjggvlesc
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_le

listing git files failed - pretending there aren't any


creating /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmpjggvlesc/tfx_user_code_Transform-0.0+158a6db728d014f2e8de973ee40665e59d06e25453dd74f1eaf238463826e91d.dist-info/WHEEL
creating '/var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmpoemwicca/tfx_user_code_Transform-0.0+158a6db728d014f2e8de973ee40665e59d06e25453dd74f1eaf238463826e91d-py3-none-any.whl' and adding '/var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmpjggvlesc' to it
adding 'keras_trainer.py'
adding 'module.py'
adding 'module_test.py'
adding 'transform.py'
adding 'tfx_user_code_Transform-0.0+158a6db728d014f2e8de973ee40665e59d06e25453dd74f1eaf238463826e91d.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+158a6db728d014f2e8de973ee40665e59d06e25453dd74f1eaf238463826e91d.dist-info/WHEEL'
adding 'tfx_user_code_Transform-0.0+158a6db728d014f2e8de973ee40665e59d06e25453dd74f1eaf238463826e91d.dist-info/top_level.txt'
adding 'tfx_user_code_Transform-0.0+158a6db728d014f2e8de973ee40665e59d06e25453dd74f1eaf238463826e91d.di

INFO:tensorflow:Assets written to: ../tfx/Transform/transform_graph/14/.temp_path/tftransform_tmp/ea057a09a08549748cb7c896a4790996/assets


INFO:tensorflow:Assets written to: ../tfx/Transform/transform_graph/14/.temp_path/tftransform_tmp/ea057a09a08549748cb7c896a4790996/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 14
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 77
        type_id: 22
        uri: "../tfx/Transform/transform_graph/14"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 78
        type_id: 14
        uri: "../tfx/Transform/transformed_examples/14"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 79
        type_id: 23
        uri: "../tfx/Transform/updated_analyzer_cache/14"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 80
        type_id: 18
        uri: "../tfx/Transform/pre_transform_schema/14"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }


In [24]:
trainer_file = os.path.join(os.getcwd(), '../components/module.py')

In [25]:
from tfx.proto import trainer_pb2

TRAINING_STEPS = 1000
EVALUATION_STEPS = 100

trainer = tfx.components.Trainer(
    module_file=trainer_file,
    examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying keras_trainer.py -> build/lib
copying module_test.py -> build/lib
copying transform.py -> build/lib
copying module.py -> build/lib
installing to /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmphkssc_tj
running install
running install_lib
copying build/lib/keras_trainer.py -> /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmphkssc_tj
copying build/lib/module_test.py -> /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmphkssc_tj
copying build/lib/transform.py -> /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmphkssc_tj
copying build/lib/module.py -> /var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/tmphkssc_tj
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt


listing git files failed - pretending there aren't any
I1123 10:02:18.241540 506195456 rdbms_metadata_access_object.cc:686] No property is defined for the Type
I1123 10:02:18.243752 506195456 rdbms_metadata_access_object.cc:686] No property is defined for the Type


Processing /Users/hannes/opensource/building-machine-learning-pipelines/tfx/_wheels/tfx_user_code_Trainer-0.0+158a6db728d014f2e8de973ee40665e59d06e25453dd74f1eaf238463826e91d-py3-none-any.whl
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
consumer_complaint_narrative_xf [(None, 1)]          0                                            
__________________________________________________________________________________________________
tf.reshape (TFOpLambda)         (None,)              0           consumer_complaint_narrative_xf[0
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 512)          256797824   tf.reshape[0][0]                 
__________________________________________________________________________________________________
r

2021-11-23 10:03:09.358554: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-23 10:03:09.358631: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-23 10:03:09.360027: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


   3/1000 [..............................] - ETA: 12:01 - loss: 0.7645 - binary_accuracy: 0.1771 - true_positives: 29.0000

...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:Assets written to: ../tfx/Trainer/model/15/Format-Serving/assets


INFO:tensorflow:Assets written to: ../tfx/Trainer/model/15/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 15
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 85
        type_id: 25
        uri: "../tfx/Trainer/model/15"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 86
        type_id: 26
        uri: "../tfx/Trainer/model_run/15"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Load TensorBoard

In [26]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri

In [27]:
log_dir = os.path.join(model_artifact_dir, 'logs/')

%load_ext tensorboard
%tensorboard --logdir {log_dir} 

### Evaluate the model

In [28]:
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing),
)
context.run(model_resolver)

ExecutionResult(
    component_id: Resolver
    execution_id: 16
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [32]:
# nb it always blesses on first run even if below threshold
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name="serving_default",
            label_key="consumer_disputed",
            # preprocessing_function_names=["transform_features"],
        )
    ],
    slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=["product"])],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.65}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value': -1e-10})
                    ),
                ),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="AUC"),
            ],
        )
    ],
)


In [33]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)
context.run(evaluator)


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x187cc8670> and <keras.engine.input_layer.InputLayer object at 0x188a8b160>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x187cc8670> and <keras.engine.input_layer.InputLayer object at 0x188a8b160>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x187c4c610> and <keras.engine.input_layer.InputLayer object at 0x1868de970>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x187c4c610> and <keras.engine.input_layer.InputLayer object at 0x1868de970>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x18b35eca0> and <keras.engine.input_layer.InputLayer object at 0x18b36fa00>).


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x192ace490> and <keras.engine.input_layer.InputLayer object at 0x192a22d30>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x192ace490> and <keras.engine.input_layer.InputLayer object at 0x192a22d30>).


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 18
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 89
        type_id: 29
        uri: "../tfx/Evaluator/evaluation/18"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 29
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 90
        type_id: 30
        uri: "../tfx/Evaluator/blessing/18"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "../tfx/Trainer/model/15"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 85
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0.dev20211102"
          }
        }
        state: LIVE
        , artifact_type: id: 30
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [34]:
#NB TFMA visualizations will not run in Jupyter Lab
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

In [35]:
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [ ]:
from tfx.proto import pusher_pb2

_serving_model_dir = "../interactive-pipeline/serving_model_dir"

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 73
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: ../tfx/Pusher/pushed_model/73, id: 88)]
        ))

### Extra stuff

In [ ]:
!mkdir -p ../interactive-pipeline/serving_model_dir

In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
  
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
  
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix

# THEN REFRESH BROWSER PAGE!

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

In [ ]:
# Show data sliced by product
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='product')

In [ ]:
# fairness indicators direct from pipeline
# https://colab.research.google.com/github/tensorflow/fairness-indicators/blob/master/fairness_indicators/examples/Fairness_Indicators_Lineage_Case_Study.ipynb